In [1]:
import os

os.chdir('/app')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



In [3]:
from dataclasses import dataclass

# config
@dataclass
class TrainConfig:
    batch_size: int = 64
    test_batch_size: int = 1000
    epochs: int = 10
    lr: float = 1e-3
    momentum: float = 0.5
    seed: int = 1
    log_interval: int = 10
    save_model: bool = False

config = TrainConfig()

In [4]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

/usr/local/lib/python3.11/dist-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
# VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # encoder
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20) # mu
        self.fc22 = nn.Linear(400, 20) # logvar

        # decoder
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)
    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [6]:
# initialize model
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [7]:
# loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [10]:
# 学習ループ
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item() / len(data):.6f}')
    
    print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')


In [9]:
from PIL import Image

def make_grid(images, rows, cols):
    w, h = images[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, image in enumerate(images):
        grid.paste(image, box=(i%cols*w, i//cols*h))
    return grid

# VAEで画像を生成し、保存する関数
def generate_and_save_images(model, epoch, output_dir, n_images=16):
    model.eval()
    with torch.no_grad():
        # ランダムな潜在ベクトルから画像を生成
        sample = torch.randn(n_images, 20).to(device)
        generated = model.decode(sample).cpu()

    # 生成された画像をPIL Imageに変換
    images = []
    for img in generated:
        img = img.view(28, 28).numpy()
        img = (img * 255).astype('uint8')
        images.append(Image.fromarray(img).convert('RGB'))

    # 画像グリッドの作成
    image_grid = make_grid(images, rows=4, cols=4)

    # 画像の保存
    os.makedirs(output_dir, exist_ok=True)
    image_grid.save(f"{output_dir}/{epoch:04d}.png")

    print(f"Saved generated images for epoch {epoch}")



In [12]:
# モデルの学習と画像生成
output_dir = "./vae_samples"
for epoch in range(1, config.epochs + 1):
    train(epoch)
    generate_and_save_images(model, epoch, output_dir)

print("Training and image generation complete")

Train Epoch: 1 [0/60000 (0%)]	Loss: 548.512451
Train Epoch: 1 [6400/60000 (11%)]	Loss: 195.243912
Train Epoch: 1 [12800/60000 (21%)]	Loss: 156.783890
Train Epoch: 1 [19200/60000 (32%)]	Loss: 145.689590
Train Epoch: 1 [25600/60000 (43%)]	Loss: 133.493103
Train Epoch: 1 [32000/60000 (53%)]	Loss: 130.844238
Train Epoch: 1 [38400/60000 (64%)]	Loss: 123.844788
Train Epoch: 1 [44800/60000 (75%)]	Loss: 122.093857
Train Epoch: 1 [51200/60000 (85%)]	Loss: 122.971054
Train Epoch: 1 [57600/60000 (96%)]	Loss: 121.206657
====> Epoch: 1 Average loss: 147.1474
Saved generated images for epoch 1
Train Epoch: 2 [0/60000 (0%)]	Loss: 128.654922
Train Epoch: 2 [6400/60000 (11%)]	Loss: 121.855042
Train Epoch: 2 [12800/60000 (21%)]	Loss: 111.146515
Train Epoch: 2 [19200/60000 (32%)]	Loss: 117.289566
Train Epoch: 2 [25600/60000 (43%)]	Loss: 111.986160
Train Epoch: 2 [32000/60000 (53%)]	Loss: 115.421242
Train Epoch: 2 [38400/60000 (64%)]	Loss: 115.905342
Train Epoch: 2 [44800/60000 (75%)]	Loss: 113.032608
Tra

In [11]:
# モデルの保存
torch.save(model.state_dict(), "vae_mnist.pth")